In [1]:
import os
import json
import numpy as np
import pandas as pd
from collections import defaultdict
from copy import deepcopy

In [5]:
# filenames = ["annotations_train.json", "annotations_val.json", "annotations_test.json"]

#######################################################################################
############################# CHANGE FILENAMES HERE ###################################
#######################################################################################
filenames = ["train_001.json", "train_005.json", "train_010.json", "train_025.json", "train_050.json"]
dir_path = "../annotations/splits"
save_path = "../annotations/SOTA/mdetr/splits"

for filename in filenames: 
    filepath = os.path.join(dir_path, filename)
    data = json.load(open(filepath))
    images = data["images"]
    new_images = []
    tokens = defaultdict(str)
    for image in images: 
        caption = image["caption"]
        image["dataset_name"] = "refcoco"
        image["tokens_negative"] = [[0, len(caption)]]
        tokens[image["id"]] = image["tokens_negative"]
        new_images.append(image)

    data["images"] = new_images

    for annotations in data["annotations"]:
        annotations["tokens_positive"] = tokens[annotations["image_id"]]
    
    new_filename = filename.split(".")[0] + "_mdetr.json"
    new_filepath = os.path.join(save_path, new_filename)
    data.pop("input_oracle_annotations")
    if "segmentation_images" in data.keys():
        data.pop("segmentation_images")
    json.dump(data, open(new_filepath, "w"))


In [4]:
test_filename = "annotations_test_mdetr.json"
test_filepath = os.path.join(save_path, test_filename)

test_data = json.load(open(test_filepath))


ablation = ["o+d+", "o+d-", "o-d+", "o-d-"]

for a in ablation: 
    new_images = []
    new_annotations = []
    tokens = defaultdict(str)
    ablation_data = deepcopy(test_data)

    for image in ablation_data["images"]:

        if a == "o+d+": 
            caption = image["caption"]
        elif a == "o+d-":
            caption = " ".join(image["caption"].split(" ")[1:])
        elif a == "o-d+":
            caption = image["caption"].split(" ")[0]
        elif a == "o-d-":
            caption = "  "

        image["caption"] = caption 
        image["tokens_negative"] = [[0, len(caption)]]
        tokens[image["id"]] = image["tokens_negative"]
        new_images.append(image)
    
    for annotation in ablation_data["annotations"]:
        annotation["tokens_positive"] = tokens[annotations["image_id"]]
        new_annotations.append(annotation)

    ablation_data["images"] = new_images
    ablation_data["annotations"] = new_annotations

    ablation_filename = test_filename.split(".")[0] + "_" + a + ".json"
    ablation_dir_path = os.path.join(dir_path, "ablation/mdetr")
    ablation_filepath = os.path.join(ablation_dir_path, ablation_filename)

    json.dump(ablation_data, open(ablation_filepath, "w"))

FileNotFoundError: [Errno 2] No such file or directory: '../annotations/splits\\SOTA/mdetr/splits\\annotations_test_mdetr.json'